In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVC


In [2]:
df = pd.read_csv(r'..\data\raw\AgeDataset-V1.csv')
df = df[df.Gender.notna()].drop(['Id','Short description'],axis=1)


In [3]:
m= df[df['Gender']=='Male' ]
f= df[df['Gender']=='Female']
df=pd.concat([m,f],ignore_index=True)
df.head()

,Name,Gender,Country,Occupation,Birth year,Death year,Manner of death,Age of death
0,George Washington,Male,United States of America; Kingdom of Great Bri...,Politician,1732,1799.0,natural causes,67.0
1,Douglas Adams,Male,United Kingdom,Artist,1952,2001.0,natural causes,49.0
2,Abraham Lincoln,Male,United States of America,Politician,1809,1865.0,homicide,56.0
3,Wolfgang Amadeus Mozart,Male,Archduchy of Austria; Archbishopric of Salzburg,Artist,1756,1791.0,NaN,35.0
4,Ludwig van Beethoven,Male,Holy Roman Empire; Austrian Empire,Artist,1770,1827.0,NaN,57.0


In [4]:
df=df.dropna().reset_index().drop('index',axis=1)

In [5]:
df['Occupation']=[x.split(';')[0] for x in df.Occupation.values]


In [6]:
count=df.Occupation.value_counts()
count

Artist                18026
Politician             7282
Athlete                5656
Researcher             2719
Military personnel     2115
                      ...  
Hauptsturmführer          1
Assessor                  1
Aerobatics                1
Co-driver                 1
Suffragist                1
Name: Occupation, Length: 796, dtype: int64

In [7]:
lista=count.index[count.values>10]
lista

Index(['Artist', 'Politician', 'Athlete', 'Researcher', 'Military personnel',
       'Journalist', 'Businessperson', 'Religious figure', 'Teacher',
       'Engineer',
       ...
       'Author', 'Mangaka', 'Impresario', 'Spree killer', 'Dentist',
       'Bobsledder', 'Political activist', 'Stunt performer',
       'Long-distance runner', 'Scenographer'],
      dtype='object', length=137)

In [8]:
df=df[df.Occupation.isin(lista)]
df

,Name,Gender,Country,Occupation,Birth year,Death year,Manner of death,Age of death
0,George Washington,Male,United States of America; Kingdom of Great Bri...,Politician,1732,1799.0,natural causes,67.0
1,Douglas Adams,Male,United Kingdom,Artist,1952,2001.0,natural causes,49.0
2,Abraham Lincoln,Male,United States of America,Politician,1809,1865.0,homicide,56.0
4,Claude Monet,Male,France,Artist,1840,1926.0,natural causes,86.0
5,Elvis Presley,Male,United States of America,Artist,1935,1977.0,natural causes,42.0
...,...,...,...,...,...,...,...,...
45822,Sahar Khodayari,Female,Iran,Athlete,1990,2019.0,suicide,29.0
45823,Shuping Wang,Female,United States of America; People's Republic of...,Researcher,1959,2019.0,natural causes,60.0
45824,Hevrin Khalaf,Female,Syria,Politician,1984,2019.0,homicide,35.0
45825,Sara Champion,Female,United Kingdom,Researcher,1946,2000.0,natural causes,54.0


In [9]:
df['Death year']=df['Death year'].astype(int)
df['Age of death']=df['Age of death'].astype(int)
df

,Name,Gender,Country,Occupation,Birth year,Death year,Manner of death,Age of death
0,George Washington,Male,United States of America; Kingdom of Great Bri...,Politician,1732,1799,natural causes,67
1,Douglas Adams,Male,United Kingdom,Artist,1952,2001,natural causes,49
2,Abraham Lincoln,Male,United States of America,Politician,1809,1865,homicide,56
4,Claude Monet,Male,France,Artist,1840,1926,natural causes,86
5,Elvis Presley,Male,United States of America,Artist,1935,1977,natural causes,42
...,...,...,...,...,...,...,...,...
45822,Sahar Khodayari,Female,Iran,Athlete,1990,2019,suicide,29
45823,Shuping Wang,Female,United States of America; People's Republic of...,Researcher,1959,2019,natural causes,60
45824,Hevrin Khalaf,Female,Syria,Politician,1984,2019,homicide,35
45825,Sara Champion,Female,United Kingdom,Researcher,1946,2000,natural causes,54


In [10]:

df['Country']=[x.split(';')[0]for x in df.Country.values]
df

,Name,Gender,Country,Occupation,Birth year,Death year,Manner of death,Age of death
0,George Washington,Male,United States of America,Politician,1732,1799,natural causes,67
1,Douglas Adams,Male,United Kingdom,Artist,1952,2001,natural causes,49
2,Abraham Lincoln,Male,United States of America,Politician,1809,1865,homicide,56
4,Claude Monet,Male,France,Artist,1840,1926,natural causes,86
5,Elvis Presley,Male,United States of America,Artist,1935,1977,natural causes,42
...,...,...,...,...,...,...,...,...
45822,Sahar Khodayari,Female,Iran,Athlete,1990,2019,suicide,29
45823,Shuping Wang,Female,United States of America,Researcher,1959,2019,natural causes,60
45824,Hevrin Khalaf,Female,Syria,Politician,1984,2019,homicide,35
45825,Sara Champion,Female,United Kingdom,Researcher,1946,2000,natural causes,54


In [22]:
X=df[['Gender','Country','Occupation','Birth year','Manner of death']]
y=df[['Death year']]

In [26]:
X

,Gender,Country,Occupation,Birth year,Manner of death
0,Male,United States of America,Politician,1732,natural causes
1,Male,United Kingdom,Artist,1952,natural causes
2,Male,United States of America,Politician,1809,homicide
4,Male,France,Artist,1840,natural causes
5,Male,United States of America,Artist,1935,natural causes
...,...,...,...,...,...
45822,Female,Iran,Athlete,1990,suicide
45823,Female,United States of America,Researcher,1959,natural causes
45824,Female,Syria,Politician,1984,homicide
45825,Female,United Kingdom,Researcher,1946,natural causes


In [24]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [34]:
def data_preparatio(X):
    Gender_enc = OrdinalEncoder()
    Country_enc = OrdinalEncoder()
    Occupation_enc = OrdinalEncoder()
    death_enc = OrdinalEncoder()
    X['Gender'] = Gender_enc.fit_transform(X[['Gender']])
    X['Country'] =Country_enc.fit_transform(X[['Country']])
    X['Occupation'] = Occupation_enc.fit_transform(X[['Occupation']])
    X['Manner of death'] = death_enc.fit_transform(X[['Manner of death']])
    return X

In [38]:
X_train=data_preparatio(X_train)
X_test=data_preparatio(X_test)


### Regressione lineae sklear

In [ ]:
linear_model= LinearRegression()
grid = {
    'fit_intercept':[True,False]
}
grid_search =GridSearchCV(estimator=linear_model,param_grid=grid_linear,cv=5,verbose=2,return_train_score=True)